# 0. Install Dependencies

In [1]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2

# 1. Test Random Environment with OpenAI Gym

In [2]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [3]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = 38 + random.randint(-3,3)
        # Reset shower time
        self.shower_length = 60 
        return self.state
    

In [4]:
env = ShowerEnv()

/Users/chesterbr/.pyenv/versions/3.7.17/lib/python3.7/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [5]:
env.observation_space.sample()

array([75.868416], dtype=float32)

In [6]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-14
Episode:2 Score:-60
Episode:3 Score:-58
Episode:4 Score:-40
Episode:5 Score:-22
Episode:6 Score:2
Episode:7 Score:-22
Episode:8 Score:-30
Episode:9 Score:-30
Episode:10 Score:-52


# 2. Create a Deep Learning Model with Keras

In [7]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [8]:
states = env.observation_space.shape
actions = env.action_space.n

In [9]:
actions

3

In [10]:
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [11]:
if 'model' in vars() or 'model' in globals():
    del model

In [12]:
model = build_model(states, actions)

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 24)                48        
_________________________________________________________________
dense_1 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 75        
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


# 3. Build Agent with Keras-RL

In [14]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [15]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [16]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

2023-10-12 16:35:24.162809: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-12 16:35:24.176050: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f9a2fcd7c00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-10-12 16:35:24.176066: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


Training for 50000 steps ...
Interval 1 (0 steps performed)
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
    1/10000 [..............................] - ETA: 4:49 - reward: 1.0000

/Users/chesterbr/.pyenv/versions/3.7.17/lib/python3.7/site-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 43s 4ms/step - reward: -0.5442
166 episodes - episode_reward: -32.566 [-60.000, 20.000] - loss: 1.204 - mae: 7.194 - mean_q: -10.023

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 44s 4ms/step - reward: -0.4894
167 episodes - episode_reward: -29.437 [-60.000, 32.000] - loss: 2.415 - mae: 11.100 - mean_q: -15.926

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 44s 4ms/step - reward: -0.4608
167 episodes - episode_reward: -27.677 [-60.000, 36.000] - loss: 2.503 - mae: 11.269 - mean_q: -16.196

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 45s 4ms/step - reward: -0.4748
166 episodes - episode_reward: -28.687 [-60.000, 40.000] - loss: 2.414 - mae: 11.113 - mean_q: -15.969

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 45s 5ms/step - reward: -0.4186
done, took 220.962 seconds


In [17]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: -58.000, steps: 60
Episode 2: reward: -54.000, steps: 60
Episode 3: reward: -60.000, steps: 60
Episode 4: reward: -60.000, steps: 60
Episode 5: reward: -54.000, steps: 60
Episode 6: reward: -56.000, steps: 60
Episode 7: reward: -60.000, steps: 60
Episode 8: reward: -58.000, steps: 60
Episode 9: reward: -60.000, steps: 60
Episode 10: reward: -56.000, steps: 60
Episode 11: reward: -54.000, steps: 60
Episode 12: reward: -60.000, steps: 60
Episode 13: reward: -56.000, steps: 60
Episode 14: reward: -60.000, steps: 60
Episode 15: reward: -54.000, steps: 60
Episode 16: reward: -60.000, steps: 60
Episode 17: reward: -56.000, steps: 60
Episode 18: reward: -58.000, steps: 60
Episode 19: reward: -60.000, steps: 60
Episode 20: reward: -54.000, steps: 60
Episode 21: reward: -54.000, steps: 60
Episode 22: reward: -60.000, steps: 60
Episode 23: reward: -60.000, steps: 60
Episode 24: reward: -60.000, steps: 60
Episode 25: reward: -54.000, steps: 60
Episo

In [18]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

Testing for 15 episodes ...


TypeError: render() got an unexpected keyword argument 'mode'

# 4. Reloading Agent from Memory

In [ ]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [ ]:
del model
del dqn
del env

In [ ]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [ ]:
dqn.load_weights('dqn_weights.h5f')

In [ ]:
_ = dqn.test(env, nb_episodes=5, visualize=True)